In [5]:
import re, os, shutil
from pathlib import Path

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from pydrive.auth import ServiceAccountCredentials

import settings

settings.init()

In [6]:
settings.NOTEBOOKS_TMPDIR

'/home/s016855141/workspace/guia_anuros_chapada/build/notebooks/tmp'

In [7]:
TEMP_DIR = f"{settings.NOTEBOOKS_TMPDIR}/textos-perfil"


In [8]:
SCOPES = ['https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_KEY_FILE = f'{Path.home()}/workspace/.sacreds/guia-anuros_service_account_key.json'

gauth = GoogleAuth()
gauth.credentials = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_KEY_FILE,SCOPES)
drive = GoogleDrive(gauth)

PROJECT_ID = '0B0M5IL0AEOXidHZmTjZzMHlLLWc'
ROOT_ID = drive.ListFile({'q': f"title contains 'dados_guia_digital' and '{PROJECT_ID}' in parents"}).GetList()[0]['id']
DIRECTORY_ESPECIES_ID = drive.ListFile({'q': f"title contains 'especies_perfil_textos' and '{ROOT_ID}' in parents"}).GetList()[0]['id']
DIRECTORY_FAMILIAS_ID = drive.ListFile({'q': f"title contains 'familias_perfil_textos' and '{ROOT_ID}' in parents"}).GetList()[0]['id']

In [10]:
arquivos_ignorar = ['modelo','README']

def getDocsFamilias():
    return [ f for f in drive.ListFile({'q':f"'{DIRECTORY_FAMILIAS_ID}' in parents and mimeType contains 'document'"}).GetList() 
                if f['title'] not in arquivos_ignorar ]
    
def getDocsEspecies():
    return [ f for f in drive.ListFile({'q': f"'{DIRECTORY_ESPECIES_ID}' in parents and mimeType contains 'document'"}).GetList()
                if f['title'] not in arquivos_ignorar ]
    
# Fetch all documents for each family
def fetchProfilesDocuments():    
    try: shutil.rmtree(TEMP_DIR)
    except(FileNotFoundError): pass

    os.makedirs(f'{TEMP_DIR}/especies')
    os.makedirs(f'{TEMP_DIR}/familias')
    
    for doc in getDocsFamilias():
        print(f"Baixando documento da família {doc['title']}")
        doc.GetContentFile(f"{TEMP_DIR}/familias/{doc['title'].lower()}.md", mimetype="text/plain")
        
        
    for doc in getDocsEspecies():
        print(f"Baixando documento da espécie {doc['title']}")
        doc.GetContentFile(f"{TEMP_DIR}/especies/{doc['title'].lower()}.md", mimetype="text/plain")
        
    
fetchProfilesDocuments()


Baixando documento da família bufonidae
Baixando documento da família hylidae
Baixando documento da espécie ololygon-skaios
Baixando documento da espécie dendropsophus-rubicundulus
Baixando documento da espécie dendropsophus-minutus
Baixando documento da espécie dendropsophus-melanargyreus
Baixando documento da espécie dendropsophus-jimi
Baixando documento da espécie bokermannohyla-pseudopseudis


KeyboardInterrupt: 